In [11]:
##  Llamamos a las librerias para hacer el web scrapping
import pandas as pd
import numpy as np
import random
import sys
sys.path.append("C:/Users/luisr/OneDrive/Desktop/ANBA/23_24/Lesiones")
import aux_func


In [2]:
equipos_vivos = ['ATL','BOS','BKN','CHI','CHA','CLE','DET','IND','MIA','MIL','NYK','ORL','PHI','TOR','WAS','DAL','DEN','GSW','HOU','LAC','LAL','MEM','MIN','NOP','OKC','PHO','POR','SAC','SAS','UTA']

In [3]:
 ##  Definimos de que temporada queremos los datos
season = '2022-2023'

##  Scrapeamos stats jugadores
stats_players = aux_func.scrapping_stats_players(season, 27)

Scrappeando estadisticas de jugadores. Espere...


  0%|          | 0/27 [00:00<?, ?it/s]

100%|██████████| 27/27 [03:30<00:00,  7.81s/it]


In [5]:
##  Stats Equipo -> Para saber el numero de partidos de cada equipo
stats_teams = aux_func.scrapping_stats_teams(season,'No')

Scrappeando estadisticas de equipos. Espere...


In [ ]:
##  Indice de jugadores
index = aux_func.scrapping_player_index(70)
index = index.replace("PHX","PHO")

In [13]:
##  Cogemos los que tienen equipo actualmente
index = index[index['Equipo'] != 'Agente Libre'] 

In [15]:
##  Para poder hacer merge necesitamos que los nombres esten en el mismo format
stats_players = aux_func.invert_name_format(stats_players)

In [16]:
## Merge
merged_stats = index.merge(stats_players, on = 'Jugador',how = 'left')

In [17]:
##  Seleccionamos solo las columnas que vamos a usar
final_stats = merged_stats[['Jugador','Equipo_x','PJ','MIN']]

In [ ]:
##  Reemplazamos NAs por 0
final_stats['MIN'] = final_stats['MIN'].replace(np.nan, 0)
final_stats['PJ'] = final_stats['PJ'].replace(np.nan, 0)

In [21]:
##  Para usar las stats de equipo, parseamos nombre de los equipos
stats_teams = aux_func.parse_team_names(stats_teams)

In [22]:
##  Nos falta otro merge para los partidos jugados por el equipo
merged_stats2 = final_stats.merge(stats_teams, left_on='Equipo_x', right_on='Equipo',how = 'left')

In [24]:
##  Ya tenemos el dataframe final
final_stats = merged_stats2[['Jugador','Equipo_x','PJ_x','MIN','PJ_y']]
final_stats.columns =['Jugador','Equipo','PJ_jug','MIN','PJ_Eq']


In [ ]:
##  Calculamos los pesos de cada jugador segun sus minutos
final_stats['peso'] = (final_stats['MIN'] + final_stats['PJ_Eq'] * 10)

In [27]:
##  Filtramos solo para los equipos vivos
final_stats = final_stats[final_stats['Equipo'].isin(equipos_vivos)]

In [49]:
##  Elegimos jugador a morir
dead_player = random.choices(list(final_stats['Jugador']), weights=tuple(final_stats['peso']), k=1)
dead_player

['Norman Powell']

In [50]:
## Creamos dataframe con los datos de las lesiones
lesiones = ['Quadriceps tendon','ACL','MCL','Achilles','Patellar tendon']

datos_lesiones = pd.DataFrame(lesiones)
datos_lesiones['numero'] = [3,33,5,23,6]
datos_lesiones['media'] = [305,410,434,346,413]
datos_lesiones['min'] = [212,232,194,210,198]
datos_lesiones['max'] = [371,1110,836,614,886]
datos_lesiones['stdev'] = [83,179,251,99,246]

In [51]:
##  Elegimos lesion aleatoriamente usando las prob de ocurrencia
lesion = random.choices(list(datos_lesiones[0]), weights=tuple(datos_lesiones['numero']), k=1)
lesion = lesion[0]

In [52]:
##  Para la duracion, debemos sacar un num aletorio con distrib normal centrada
##  en la media calculada

mean = datos_lesiones[datos_lesiones[0] == lesion]['media']
stdev = datos_lesiones[datos_lesiones[0] == lesion]['stdev']
minimo = float(datos_lesiones[datos_lesiones[0] == lesion]['min']) 
maximo = float(datos_lesiones[datos_lesiones[0] == lesion]['max'])

duracion = float(random.gauss(mean, stdev))
while duracion < minimo or duracion > maximo:
    duracion = float(random.gauss(mean, stdev))

duracion = round(duracion,0)



C:\Users\luisr\AppData\Local\Temp\ipykernel_18488\2550864755.py:6: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  minimo = float(datos_lesiones[datos_lesiones[0] == lesion]['min'])
C:\Users\luisr\AppData\Local\Temp\ipykernel_18488\2550864755.py:7: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  maximo = float(datos_lesiones[datos_lesiones[0] == lesion]['max'])
C:\Users\luisr\AppData\Local\Temp\ipykernel_18488\2550864755.py:9: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  duracion = float(random.gauss(mean, stdev))


In [53]:
aux_func.popupmsg('Jugador: ' + str(dead_player[0]) + '  lesion: ' + str(lesion) + '    duracion: ' + str(duracion) + ' dias')